In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [2]:
df2023_2024 = pd.read_excel("DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx")
df2022_2023 = pd.read_excel("DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx", sheet_name=None)
df2020_2021 = pd.read_excel("DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx", sheet_name=None)


# Parte 1
## A) Dimensión de la base de datos

In [ ]:
#Obtener shape de cada dataframe
print("Shape de df2023_2024: ", df2023_2024.shape)
for key in df2022_2023:
    print("Shape de df2022_2023: ", key , df2022_2023[key].shape)
for key in df2020_2021:
    print("Shape de df2020_2021: ", key , df2020_2021[key].shape)


# Parte 2
## A) Selecciona el conjunto de datos a utilizar
Decide qué conjunto de datos se utilizará. Explica por qué se incluyeron o excluyeron ciertos datos.
Identifica las columnas objetivo


Bases de datos seleccionadas: Datos historicos 2023-2024, Datos historicos 2022-2023, Datos historicos 2020-2021 y datos históricos 2021.
Estas bases se seleccionaron pues son las más recientes, completas y presentan un formato similar para su análisis. Al ser series de tiempo se pueden juntar entre ellas para hacer un análisis más completo.
Las columnas objetivo seria tener cada columna con el formato "Estación Contaminante (medida)", indexados por su fecha, de manera que se puedan almacenar los datos como una serie de tiempo durante los 4 años.

# Hacer columnas similares para cada base de datos

Preparación de 2023-2024

In [4]:
df2023_2024.loc[0,:] = df2023_2024.loc[0,:].fillna("A") #Se rellena un valor de A para que no haya problemas al juntar las entradas
df2023_2024.columns = df2023_2024.columns.str.split('.').str[0] #Se toma el nombre de la columna sin el .1 o .2
df2023_2024 = df2023_2024.replace("WDV", "WDR") # Se cambia el nombre de la columna WDV a WDR, puesto que en las demas bases de datos es WDR
df2023_2024.columns = (df2023_2024.columns + " " + df2023_2024.loc[0] + " (" + df2023_2024.loc[1] +")").str.upper() #Se junta el nombre de la medida, con la estacion
df2023_2024.drop([0,1], inplace=True) # Se eliminan las filas que contenian los nombres de las columnas
df2023_2024.rename(columns={"UNNAMED: 0 A (DATE)": "FECHA"}, inplace=True) #Se cambia el nombre de la columna de fecha
df2023_2024["FECHA"] = pd.to_datetime(df2023_2024["FECHA"]) # Se convierte la columna de fecha a datetime
#Se cambiar el nombre de las columnas para que coincidan con las demas bases de datos
df2023_2024.columns = df2023_2024.columns.str.replace("SURESTE2", "SURESTE 2")
df2023_2024.columns = df2023_2024.columns.str.replace("NORESTE2", "NORESTE 2")
df2023_2024.columns = df2023_2024.columns.str.replace("SUROESTE2", "SUROESTE 2")

Preparación de 2022-2023

In [5]:
# Se crea un diccionario con los nombres de las columnas de df2022_2023
modified_keys = {}

for key in df2022_2023.keys():
    # Se revisa si la ultima letra de la llave es un digito
    if key[-1].isdigit():
        # Se modifica la llave para que tenga un espacio entre la penultima letra y el ultimo digito
        new_key = key[:-1] + " " + key[-1]
    else:
        # De lo contrario, se deja la llave igual
        new_key = key
    # Y se guarda la llave modificada en el diccionario
    modified_keys[key] = new_key

# Se cambiar el nombre de las columnas de las bases de datos de 2022_2023 con las llaves generadas
df2022_2023 = {modified_keys[key]: df for key, df in df2022_2023.items()}
#Se cambian las unidades para que coincidan con las de df2023_2024
units = {"CO": "(PPM)", "CO2": "(PPM)", "NO": "(PPB)", "NO2": "(PPB)", "NOX": "(PPB)", "O3": "(PPB)", "PM10": "(UG/M3)", "PM2.5": "(UG/M3)", "SO2": "(PPB)", "PRS": "(MMHG)", "RH": "(%)", "TOUT": "(DEGC)", "WSP": "(M/S)", "RAINF": "(MM/HR)", "WD": "(DEG)", "WSR": "(KMPH)", "WDR": "(DEG)", "SR": "(KW/M2)"}
df2022_2023.keys()
for key in df2022_2023.keys():
    # Se agregan las unidades a las columnas que coinciden con las de df2023_2024 
    for column in df2022_2023[key].columns:
        if column in units:
            df2022_2023[key].rename(columns={column: key+ " " + column + " " + units[column]}, inplace=True)
        

#Se junta el diccionario de df2022_2023 en un solo dataframe
from functools import reduce

# Se elimina catalogo de las bases de datos
dfs_to_merge = [df2022_2023[key] for key in df2022_2023 if key != 'CATÁLOGO']

# Se juntan las bases de datos por la columna de fecha
df2022_2023_merged = reduce(lambda left, right: pd.merge(left, right, on='date',how="outer"), dfs_to_merge)

# Se cambia el nombre de la columna de fecha y se cambia el tipo de dato a datetime
df2022_2023_merged.rename(columns={"date": "FECHA"}, inplace=True)
df2022_2023_merged["FECHA"] = pd.to_datetime(df2022_2023_merged["FECHA"])

Preparación de 2021-2022

In [6]:
# Se crea un diccionario con los nombres de las columnas de df2020_2021
modified_keys = {}

for key in df2020_2021.keys():
    # Se revisa si la ultima letra de la llave es un digito
    if key[-1].isdigit():
        # Se modifica la llave para que tenga un espacio entre la penultima letra y el ultimo digito
        new_key = key[:-1] + " " + key[-1]
    else:
        # De lo contrario, se deja la llave igual
        new_key = key
    # Y se guarda la llave modificada en el diccionario
    modified_keys[key] = new_key

# Se cambia el nombre de las columnas de las bases de datos de 2020_2021 con las llaves generadas
df2020_2021 = {modified_keys[key]: df for key, df in df2020_2021.items()}

for key in df2020_2021.keys():
    # Se agregan las unidades a las columnas que coinciden con las de df2023_2024
    for column in df2020_2021[key].columns:
        if column in units:
            df2020_2021[key].rename(columns={column: key + " " + column + " " + units[column]}, inplace=True)


# Se elimina catalogo y noroeste 3 de las bases de datos
dfs_to_merge = [df2020_2021[key] for key in df2020_2021 if key not in ['CATÁLOGO', 'NOROESTE 3']]

# Merge all DataFrames on the 'date' column
df2020_2021_merged = reduce(lambda left, right: pd.merge(left, right, on='date', how="outer"), dfs_to_merge)

# Se cambia el nombre de la columna de fecha y se cambia el tipo de dato a datetime
df2020_2021_merged.rename(columns={"date": "FECHA"}, inplace=True)
df2020_2021_merged["FECHA"] = pd.to_datetime(df2020_2021_merged["FECHA"])

In [ ]:
#Dimensiones de los datasets
print("Dimensiones de df2023_2024: ", df2023_2024.shape)
print("Dimensiones de df2022_2023_merged: ", df2022_2023_merged.shape)
print("Dimensiones de df2020_2021_merged: ", df2020_2021_merged.shape)


In [8]:
#Se eliminan las columnas que no tienen al menos el 2% de los datos
df2023_2024 = df2023_2024.dropna(thresh=0.02*len(df2023_2024), axis=1)
df2022_2023_merged = df2022_2023_merged.dropna(thresh=0.02*len(df2022_2023_merged), axis=1)
df2020_2021_merged = df2020_2021_merged.dropna(thresh=0.02*len(df2020_2021_merged), axis=1)


In [ ]:
#Nuevas dimensiones de los datasets
print("Dimensiones de df2023_2024: ", df2023_2024.shape)
print("Dimensiones de df2022_2023_merged: ", df2022_2023_merged.shape)
print("Dimensiones de df2020_2021_merged: ", df2020_2021_merged.shape)


In [10]:
#Hacer las fechas datetime
df2023_2024["FECHA"] = pd.to_datetime(df2023_2024["FECHA"])
df2022_2023_merged["FECHA"] = pd.to_datetime(df2022_2023_merged["FECHA"])
df2020_2021_merged["FECHA"] = pd.to_datetime(df2020_2021_merged["FECHA"])

Unir las bases de datos por fecha

In [11]:
df2022_merged = df2022_2023_merged.query("FECHA < '2023-01-01'")

In [12]:
dfs_to_merge = [df2023_2024, df2022_merged, df2020_2021_merged]

# Merge all DataFrames on the 'date' column
df_time_serie = pd.concat([dfs_to_merge[0], dfs_to_merge[1], dfs_to_merge[2]], axis=0)

In [13]:
df_time_serie.set_index("FECHA", inplace=True)

In [14]:
#Ordenar por fecha
df_time_serie.sort_index(inplace=True)

In [ ]:
#Get the index of the maximun value of each column
max_values = df_time_serie.idxmax()
max_values

In [ ]:
#Se cambian los valores a float
df_time_serie = df_time_serie.astype(float)
#Se obtine el rango posible de valores por contaminante
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","TOUT","WSR","WDR", "SR", "PRS"]
for contaminante in contaminantes:
    print(contaminante)
    print("Min: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").min().min())
    print("Max: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").max().max())
    #get the index of the max value
    print("Fecha del maximo: ", max_values.filter(regex=rf"\b{contaminante}\b").values[0])
    #Get the estacion del maximo
    print("Estacion del maximo: ", max_values.filter(regex=rf"\b{contaminante}\b").index[0])
    print("")

In [ ]:
df_time_serie = df_time_serie.astype(float)
#Se obtine el rango posible de valores por contaminante
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    print("Min: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").min())
    print("Max: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").max())
    print("")

In [ ]:

#Se obtinen los cuantiles de los contaminantes
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","TOUT","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    print(df_time_serie.filter(regex=rf"\b{contaminante}\b").quantile([0.25,0.5,0.75]))
    print("")

In [ ]:
#Se obtiene el menor cuartil de cada contaminante en 0.25, asi como el mayor cualtil de 0.75 de cada contaminante
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","TOUT","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    print("Cuartil 0.25: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").quantile(0.25).min())
    print("Cuartil 0.75: ", df_time_serie.filter(regex=rf"\b{contaminante}\b").quantile(0.75).max())
    print("")

In [ ]:
#Promedios
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    promedio = df_time_serie.filter(regex=rf"\b{contaminante}\b").mean().mean()
    print(f"Promedio: {promedio}")
    print("")

In [ ]:
#Mediana
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    mediana = df_time_serie.filter(regex=rf"\b{contaminante}\b").median().max()
    print(f"Mediana: {mediana}")
    print("")

In [ ]:
#Moda
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    moda = df_time_serie.filter(regex=rf"\b{contaminante}\b").mode().max()
    print(f"Moda: {moda}")
    print("")

In [ ]:
#Obtener la desviacion estandar de cada contaminante
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    print(df_time_serie.filter(regex=rf"\b{contaminante}\b").std().max())
    print("")

Varianza

In [ ]:
#Obtener la varianza de cada contaminante
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2", "RAINF", "RH","SR","TOUT","PRS","WSR","WDR"]
for contaminante in contaminantes:
    print(contaminante)
    print(df_time_serie.filter(regex=rf"\b{contaminante}\b").std().max()**2)
    print("")

# Parte 2) Verifica la calidad de datos

In [70]:

df_time_serie_filtered = df_time_serie.copy()
#Eliminar las columnas de NOROESTE 3
df_time_serie_filtered = df_time_serie_filtered.drop(columns=df_time_serie_filtered.filter(regex="NOROESTE 3").columns)
df_time_serie_filtered.replace(-9999, np.nan, inplace=True)


In [71]:
# Se limpia el valor extremo de la columna de TOUT menor a -20
df_tout_columns = df_time_serie_filtered.filter(regex=rf"\bTOUT\b").columns
df_time_serie_filtered[df_tout_columns] = df_time_serie_filtered[df_tout_columns].mask(df_time_serie_filtered[df_tout_columns].lt(-20), np.nan)

# Se limpia el valor extremo de la columna de CO menor a 0
df_CO_columns = df_time_serie_filtered.filter(regex=rf"\bCO\b").columns
df_time_serie_filtered[df_CO_columns] = df_time_serie_filtered[df_CO_columns].mask(df_time_serie_filtered[df_CO_columns].lt(0), np.nan)

# Se limpia el valor extremo de la columna de NO2 menor a 0
df_NO_columns = df_time_serie_filtered.filter(regex=rf"\bNO2\b").columns
df_time_serie_filtered[df_NO_columns] = df_time_serie_filtered[df_NO_columns].mask(df_time_serie_filtered[df_NO_columns].lt(0), np.nan)

# Se limpia el valor extremo de la columna de NO2 menor a 0
df_SR_columns = df_time_serie_filtered.filter(regex=rf"\bSR\b").columns
df_time_serie_filtered[df_SR_columns] = df_time_serie_filtered[df_SR_columns].mask(df_time_serie_filtered[df_SR_columns].lt(0), np.nan)



In [72]:
#Hacer función para reemplazar valor mayores a cierta cantidad, dado un contaminante y un dataframe
def replace_outliers(contaminante,cantidad, df):
    df_columns = df.filter(regex=rf"\b{contaminante}\b").columns
    df[df_columns] = df[df_columns].mask(df[df_columns].gt(cantidad), np.nan)

def max_std(contaminante, df):
    return df.filter(regex=rf"\b{contaminante}\b").std().max() * 3
# moda = df_time_serie.filter(regex=rf"\b{contaminante}\b").std().max()
contaminantes = ["CO", "NO", "NO2", "NOX", "O3", "PM10", "PM2.5", "SO2"]
    
for contaminante in contaminantes:
    replace_outliers(contaminante, max_std(contaminante, df_time_serie_filtered), df_time_serie_filtered)

replace_outliers("RAINF", 800, df_time_serie)
replace_outliers("RH", 30, df_time_serie)
replace_outliers("SR", 5, df_time_serie)
replace_outliers("TOUT", 60, df_time_serie)
replace_outliers("PRS", 900, df_time_serie)
replace_outliers("WSR", 300, df_time_serie)
replace_outliers("WDR", 360, df_time_serie)

In [73]:
df_time_serie_2022 = df_time_serie_filtered.query("FECHA >= '2022-01-01'")

# Graficos sin outliers raros

In [ ]:
CO_columns = df_time_serie_2022.filter(regex=rf"\bPM10\b").columns.tolist()
sns.set(style="whitegrid", palette="muted")
fig, ax = plt.subplots(figsize=(15, 10))
sns.boxplot(data=df_time_serie_2022[CO_columns], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=85)
ax.set_title("PM10 Levels Across Different Regions", fontsize=16, weight='bold')
ax.set_ylabel("PM10 Levels (PPM)", fontsize=14)
ax.set_xlabel("Regions", fontsize=14)
sns.despine(trim=True)
plt.tight_layout()
plt.savefig("PM10_levels.png", dpi = 400)
plt.show


In [ ]:
# Boxplots
CO_columns = df_time_serie_2022.filter(regex=rf"\bCO\b").columns.tolist()
df_time_serie_2022.boxplot(column=CO_columns, figsize=(15, 10), rot=85)
plt.show()

In [ ]:
# Se hace un histograma por contaminante agrupado por estación
for contaminante in contaminantes:
    fig, ax = plt.subplots(figsize=(15, 10))
    df_time_serie_2022.filter(regex=rf"\b{contaminante}\b").hist(ax=ax, bins=40)
    plt.title(contaminante)
    plt.savefig(f"{contaminante}_histogram.png", dpi = 400)
    plt.show()

In [77]:
estaciones = ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NORTE', 'SUR']

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
# Quitar la palabra 'SURESTE 2' de cada histograma individual

df_time_serie_2022.filter(regex=rf"\b{'SURESTE 2'}\b").hist(ax=ax, bins=40)
plt.title('SURESTE 2')
plt.savefig(f"{'SURESTE 2'}_histogram.png", dpi = 400)
plt.show()

In [ ]:
# Se hace un histograma por contaminante agrupado por estación
for estacion in estaciones:
    fig, ax = plt.subplots(figsize=(15, 10))
    df_time_serie_2022.filter(regex=rf"\b{estacion}\b").hist(ax=ax, bins=40)
    plt.title(estacion)
    plt.savefig(f"{estacion}_histogram.png", dpi = 400)
    plt.show()

In [ ]:
#Obtener un histograma de los contaminantes en cada estación
plt.figure(figsize=(15, 10))
sns.histplot(data=df_time_serie_2022.filter(regex=rf"\bSURESTE\b"), bins=40)
plt.title("SURESTE")
plt.show()

# Heat map

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df_time_serie_2022.filter(regex=rf"\bPM10\b").corr('spearman'), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.tight_layout()
plt.savefig("PM2.5_heatmap.png", dpi = 400)
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df_time_serie_2022.filter(regex="SUROESTE 2").corr('spearman'), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.tight_layout()
plt.savefig("PM2.5_heatmap.png", dpi = 400)
plt.show()

In [83]:
# Create the entry filled with NaNs
df_time_serie_2022.loc['2023-02-11 19:00:00'] = np.nan
df_time_serie_2022.loc['2023-10-31 17:00:00'] = np.nan
#Set the index to datetime
df_time_serie_2022.index = pd.to_datetime(df_time_serie_2022.index) 
# Mes
df_time_serie_2022["Mes"] = df_time_serie_2022.index.month
#Day of year
df_time_serie_2022["Dia"] = df_time_serie_2022.index.dayofyear
#Hour
df_time_serie_2022["Hora"] = df_time_serie_2022.index.hour

In [84]:
df_time_series_2022 = df_time_serie_2022.asfreq('h',fill_value=np.nan)

In [85]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=7, weights="distance")
df_time_serie_2022_nonans = imputer.fit_transform(df_time_serie_2022)



In [87]:
# Transform the numpy array back to a DataFrame with the same columns

df_time_serie_2022_nonans = pd.DataFrame(df_time_serie_2022_nonans, columns=df_time_serie_2022.columns, index=df_time_serie_2022.index)

In [88]:
df_time_serie_2022_nonans.to_csv("df_time_serie_2022.csv")